「C++でのTorchScriptモデルのロード手法」
======================================
【原題】LOADING A TORCHSCRIPT MODEL IN C++

【原著】記載なし

【元URL】https://pytorch.org/tutorials/advanced/cpp_export.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2020年10月27日

【チュトーリアル概要】

本チュートリアルでは、TorchScriptを利用して、C++環境でモデルを実行可能に変換する手順と、実際にC++で実行する方法について解説します。


---

PyTorchという名前の通り、PyTorchのメインのインターフェースはPythonです。

Pythonは柔軟で手軽に使用しやすいプログラミング言語ですが、Pythonのこれらの特性が好ましくない状況もあります。

Pythonの適用が難しい環境の一つがプロダクション、製品です。

製品では低レイテンシと厳しいデプロイメント要件が求められます。



プロダクション、製品の場合にはC++ が選択されるケースが多いです（Java、Rust、Goなどの他の言語と併用する場合も含める）。

本チュートリアルでは、既存の Pythonモデルから、Python環境に依存することなく、C++環境でロードして実行できる、シリアライズされた表現（モデル）へと変換するする手順の概要を説明します。


Step 1: PyTorch Model を Torch Scriptに変換する
=====

---
PyTorchモデルのPythonからC++への変換は、Torch Scriptを利用することで可能になります。

Torch Scriptは、Torch Scriptコンパイラによって解釈、コンパイル、シリアライズ化されたPyTorchモデルの表現手法の一種です。

<br>

 "eager" API で書かれた既存の PyTorch モデルで作成している場合は、まずモデルを Torch Script に変換する必要があります。

（日本語訳注： "eager" APIとは、いわゆる普通のPyTorchの使い方だと思っていください。Define by Runのことです。）





一般的なケースでは、後ほど解説するように、この変換はほんの少しの実装で実現できます。

すでに Torch Scriptのモジュールを用意できている場合は、本チュートリアルの次のセクションに進んでください。



PyTorchモデルをTorch Scriptに変換するには、2つの方法があります。

<br>

1つ目はトレース（tracing）と呼ばれる方法です。モデルにサンプル入力を与え、一度推論計算を実施し、その入力から出力までの計算の流れをモデルに記録することで、モデルの構造を把握する仕組みです。

この方法は、制御フローの利用（if文、for文など）が限られているモデルに適しています。

<br>

2つ目の手法は、Torch Script コンパイラがモデルコードを直接解析してコンパイルすることができるように、モデルに明示的なアノテーションを追加する方法、スクリプトです（scripting）。



【ポイント】

上記の2つ手法についての、正確な説明や内容、使い分けについては、[Torch Script reference](https://pytorch.org/docs/master/jit.html)も参考にしてください。

**tracingによるTorch Scriptへの変換**


tracingを使ってPyTorchモデルをTorch Scriptに変換するには、モデルのインスタンスと入力データの例を、torch.jit.trace関数に渡します。

これにより、モデルの順伝搬の計算の流れをモジュールのフォワードメソッドとして用意した torch.jit.ScriptModule オブジェクトが作成できます。

In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

tracingで作られたモジュール（ScriptModule）は、通常のPyTorchモジュールと同じように使用できます。

In [3]:
output = traced_script_module(torch.ones(1, 3, 224, 224))
output[0, :5]

tensor([ 0.9378, -0.6521,  0.1187,  0.7416, -0.3259], grad_fn=<SliceBackward>)

（日本語訳注：英語版チュートリアルページとは結果が異なります。訓練済みモデルがアップデートされたのかもしれません）


**scriptingによるTorch Scriptへの変換**

モデルの順伝搬が、特定の形式の制御フロー（if文やfor文など）を使用しているなど、特定の状況下の場合には、Torch Scriptで直接モデルを記述します。

例えば、以下のような通常のPytorchモデルがあると仮定します。


In [4]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

このモジュールの順伝搬（フォワードメソッド）は入力に依存した制御フロー（if文）を使用しているため、tracingでの変換には適していません。

ですが、scrptingで、ScriptModuleに変換することができます。

ScriptModuleに変換するには、以下のようにtorch.jit.scriptでコンパイルする必要があります。

In [5]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

nn.Moduleの中で、TorchScriptがまだサポートしていないPythonの機能を使っているメソッドを除外する必要がある場合は、``@torch.jit.ignore``のアノテーションを付けておきます。

変数``my_module`` は、シリアライズ化されたScriptModule のインスタンスです。

（日本語訳注：英語版では、変数``my_module``と記載されているのですが、正しくは、変数``sm``がScriptModuleのインスタンスです。）


Step 2:シリアライズ化されたScriptモジュールをファイルに保存する
=====

---
tracingや、PyTorchモデルのscriptingでScriptModuleを作成できたら、ファイルへと変換する準備は完了です。

今後はC++でこのファイルからモジュールをロードすれば、Pythonに依存せずにモデルを実行できるようになります。



先ほどのtracingの例で示した ResNet18 モデルを保存するとしましょう。

このファイルへのシリアライズ（ファイル保存）を実行するには、モジュールの [save](https://pytorch.org/docs/master/jit.html#torch.jit.ScriptModule.save) を呼び出してファイル名を渡すだけです。


In [6]:
traced_script_module.save("traced_resnet_model.pt")

In [7]:
# 日本語訳注：実際に保存されたファイル「traced_resnet_model.pt」があるか確かめてみましょう
!ls

sample_data  traced_resnet_model.pt


上記のコードにより、作業ディレクトリにtraced_resnet_model.ptファイルが生成されました。

scrptingで作成した``my_module`` をシリアライズしたい場合は、 ``my_module.save("my_module_model.pt")`` と実装してください。


Step 3: C++でScriptモジュールのファイルをロードする
=====

---
シリアライズされたPyTorchモデルをC++でロードするには、アプリケーションはPyTorch C++ API（LibTorchと呼ばれます）をincludeする必要があります。

LibTorch ディストリビューションには、共有ライブラリ、ヘッダファイル、CMakeビルド設定ファイルの一式が含まれています。

<br>

CMake は LibTorch をincludeするための必須条件ではありませんが、推奨されるアプローチであり、将来的には十分にサポートされるでしょう。

本チュートリアルでは、CMakeとLibTorchを使って、シリアル化されたPyTorchモデルをロードして実行するだけの最小限のC++アプリケーションを構築します。





**最小限のC++アプリケーション**

まず、モジュールをロードするコードについて説明します。

以下のように実装します。

（日本語訳注：以下、C++のプログラムになるので、実行はできません。C++環境でお試しください）


In [ ]:
%%writefile example-app.cpp

#include <torch/script.h> // One-stop header.

#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
  if (argc != 2) {
    std::cerr << "usage: example-app <path-to-exported-script-module>\n";
    return -1;
  }


  torch::jit::script::Module module;
  try {
    // Deserialize the ScriptModule from a file using torch::jit::load().
    module = torch::jit::load(argv[1]);
  }
  catch (const c10::Error& e) {
    std::cerr << "error loading the model\n";
    return -1;
  }

  std::cout << "ok\n";
}

<torch/script.h> ヘッダーで、サンプルの実行に必要な内容が、LibTorch ライブラリからすべてincludeされます。

このアプリケーションは、ファイルに変換されたPyTorch ScriptModuleのファイルパスをコマンドライン引数として受け取り、このファイルパスを入力として受け取るtorch::jit::load()関数を使用して、モジュールをデシリアライズします（戻す変換）。

次に、torch::jit::script::Module オブジェクトをインスタンス化しています。

モデルの実行方法については、次に解説します。





**LibTorchの導入とアプリケーションの構築**


上記のコードを example-app.cpp というファイルとして、保存したとします。

これをビルドするための最小限の CMakeLists.txt は、次のようなシンプルな内容になります。

```
cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(custom_ops)

find_package(Torch REQUIRED)

add_executable(example-app example-app.cpp)
target_link_libraries(example-app "${TORCH_LIBRARIES}")
set_property(TARGET example-app PROPERTY CXX_STANDARD 14)
```

サンプルアプリケーションをビルドするために、あと必要となるものが、 LibTorch ディストリビューションです。

PyTorch のウェブサイトの[ダウンロードページ](https://pytorch.org/)から、最新の安定版を入手することができます。

最新のアーカイブをダウンロードして解凍すると、以下のようなディレクトリ構造のフォルダが生成されます。



```
libtorch/
  bin/
  include/
  lib/
  share/
```

- lib/ フォルダには、リンクする必要がある共有ライブラリが含まれています。

- include/フォルダには、プログラムがインクルードする必要があるヘッダファイルが含まれています。

- share/フォルダには、上記のシンプルなfind_package(Torch)コマンド（上記で記述したCMakeLists.txt で利用するコマンドです）を有効にするために必要なCMakeの設定が含まれています。

【ヒント】

Windowsでは、デバッグビルドとリリースビルドはABI互換性がありません（＝Application Binary Interfaceではない）。

 デバッグモードでプロジェクトをビルドする場合は、LibTorchのデバッグバージョンを試してみてください。

また、以下の cmake --build .　の行で正しい設定を指定してください。

最後のステップはアプリケーションの構築です。

ここでは、ディレクトリが以下のようにレイアウトされていると仮定します。

```
example-app/
  CMakeLists.txt
  example-app.cpp
```

example-app/フォルダ内で、次のコマンドを実行してアプリケーションをビルドすることができます。

```
mkdir build
cd build
cmake -DCMAKE_PREFIX_PATH=/path/to/libtorch ..
cmake --build . --config Release
```

ここで、/path/to/libtorch は、解凍した LibTorch ディストリビューションへのフルパスを示します（各ユーザーの環境、LitTorchが導入されたパスに合わせてください）。

すべてがうまくいくと、以下のように表示されます。

```
root@4b5a67132e81:/example-app# mkdir build
root@4b5a67132e81:/example-app# cd build
root@4b5a67132e81:/example-app/build# cmake -DCMAKE_PREFIX_PATH=/path/to/libtorch ..
-- The C compiler identification is GNU 5.4.0
-- The CXX compiler identification is GNU 5.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE
-- Configuring done
-- Generating done
-- Build files have been written to: /example-app/build
root@4b5a67132e81:/example-app/build# make
Scanning dependencies of target example-app
[ 50%] Building CXX object CMakeFiles/example-app.dir/example-app.cpp.o
[100%] Linking CXX executable example-app
[100%] Built target example-app
```

先ほどtracingで作成したResNet18モデルのファイル、traced_resnet_model.ptへのパスを、生成されたexample-appバイナリに与えると、「ok」と返ってくるはずです。

<br>

なお、以下の例を my_module_model.pt と一緒に実行しようとすると、「入力の形状が不適合です」というエラーが出る点に注意してください。

なぜなら、my_module_model.pt は 入力が4 次元ではなく 1 次元を想定しているからです。

```
root@4b5a67132e81:/example-app/build# ./example-app
<path_to_model>/traced_resnet_model.pt
ok
```

Step 4: C++でScriptモジュールを実行する
=====

---
C++ でシリアル化された ResNet18 のロードに成功したので、あと数行で、モデルが実行できる段階まで到達しました。

以下の内容を、C++ アプリケーションの main() 関数に追加してみましょう。





```
// Create a vector of inputs.
std::vector<torch::jit::IValue> inputs;
inputs.push_back(torch::ones({1, 3, 224, 224}));

// Execute the model and turn its output into a tensor.
at::Tensor output = module.forward(inputs).toTensor();
std::cout << output.slice(/*dim=*/1, /*start=*/0, /*end=*/5) << '\n';
```

最初の2行は、モデルへの入力を設定しています

torch::jit::IValue (script::Module メソッドが受け付けて返す、型が消された値です)、 のベクトル変数を作成し、入力として追加します。

<br>

次に script::Module の forwardメソッドに作成した入力ベクトルを渡して実行します。

戻り値として新たなIValueを取得し、これを toTensor()メソッドを呼び出して、テンソルの形に変換します。


【ヒント】

torch::ones のような関数や PyTorch C++ API の一般的な使い方については、https://pytorch.org/cppdocs のドキュメントを参照してください。

 PyTorch C++ API は Python API とほぼ同等の機能を提供しており、Python と同様にテンソルを操作したり処理したりすることができます。


最後の行では、出力の最初の5つの要素を表示します。

このチュートリアルの冒頭部分でPythonでモデルに同じ入力を与えたので、理想的には同じ出力が表示されるはずです。

アプリケーションを再コンパイルして、同じシリアル化されたモデルで実行してみましょう。

```
root@4b5a67132e81:/example-app/build# make
Scanning dependencies of target example-app
[ 50%] Building CXX object CMakeFiles/example-app.dir/example-app.cpp.o
[100%] Linking CXX executable example-app
[100%] Built target example-app
root@4b5a67132e81:/example-app/build# ./example-app traced_resnet_model.pt
-0.2698 -0.0381  0.4023 -0.3010 -0.0448
[ Variable[CPUFloatType]{1,5} ]
```

参考までに、以前はPythonでの出力次の通りでした。


```python
tensor([-0.2698, -0.0381,  0.4023, -0.3010, -0.0448], grad_fn=<SliceBackward>)
```

うまく、一致しています！

（日本語訳注：英語のチュートリアルページとは結果が異なります。訓練済みモデルがアップデートされた影響と思われます。。）


【ヒント】

モデルをGPUメモリに移動し、GPU上で実行するには、model.to(at::kCUDA);と実装します。

tensor.to(at::kCUDA)を呼び出すことで、モデルへの入力データもCUDAメモリ内に新たに作られます。


Step 5: ヘルプページとAPIの解説
=====

---
本チュートリアルを通して、Python から C++環境へ PyTorchモデルを変換する手順を、概念的に理解していただけたかと思います。

本チュートリアルで説明した内容を使えば、素の“eager”モードのPyTorchモデルから、

- コンパイルされたScriptModule（Python環境で動作）
- ディスク上に保存されたファイル
- C++で実行可能なscript::Module

へと、変換することができます。



もちろん、本チュートリアルでは説明しきれなかった概念もたくさんあります。

<br>

たとえば、C++ や CUDA で実装されたカスタム演算子で ScriptModuleを拡張し、そのカスタム演算子を、純粋なC++の本番環境で読み込んで、ScriptModule内で実行したいと思うかもしれません。

このようなことは可能であり、実際にサポートされているという朗報があります。

今のところは、[このフォルダ](https://github.com/pytorch/pytorch/tree/master/test/custom_operator)からサンプルを探してみてください。

その他、以下のリンク先の情報が役立つでしょう。

- The Torch Script のリファレンス: https://pytorch.org/docs/master/jit.html
- The PyTorch C++ APIのドキュメント: https://pytorch.org/cppdocs/
- The PyTorch Python APIのドキュメント: https://pytorch.org/docs/



何か問題が発生したり質問がある場合は、[フォーラム](https://discuss.pytorch.org/)や [GitHubのIssue](https://github.com/pytorch/pytorch/issues)を活用することもできます。